<a href="https://colab.research.google.com/github/BenShieh233/Learn_Python/blob/main/Pricing%20Scenarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set-Up

In [1]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo. 
#Uncomment the appropriate solver that you need.
#for reference, see https://colab.research.google.com/drive/1yGk8RB5NXrcx9f1Tb-oCiWzbxh61hZLI?usp=sharing

#installing and importing pyomo
!pip install -q pyomo
from pyomo.environ import *

###installing and importing specific solvers (uncomment the one(s) you need)
###glpk
!apt-get install -y -qq glpk-utils
###cbc
#!apt-get install -y -qq coinor-cbc
###ipopt
#!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
#!unzip -o -q ipopt-linux64
###bonmin
#!wget -N -q "https://ampl.com/dl/open/bonmin/bonmin-linux64.zip"
#!unzip -o -q bonmin-linux64
###couenne
#!wget -N -q "https://ampl.com/dl/open/couenne/couenne-linux64.zip"
#!unzip -o -q couenne-linux64
###geocode
#!wget -N -q "https://ampl.com/dl/open/gecode/gecode-linux64.zip"
#!unzip -o -q gecode-linux64

#Using the solvers:
#SolverFactory('glpk', executable='/usr/bin/glpsol')
#SolverFactory('cbc', executable='/usr/bin/cbc')
#SolverFactory('ipopt', executable='/content/ipopt')
#SolverFactory('bonmin', executable='/content/bonmin')
#SolverFactory('couenne', executable='/content/couenne')
#SolverFactory('gecode', executable='/content/gecode')

     |████████████████████████████████| 11.1 MB 4.2 MB/s 
     |████████████████████████████████| 49 kB 3.4 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 124015 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up

In [2]:
import pandas as pd

#Code that solves one instance of the Markdown Pricing Problem

In [3]:
#inputs
list_price = 55.48 #undiscounted price
demand = [120, 160, 194, 223] #for 0%, 15%, 30%, 50% discounts, respectively
weeksleft = 15 #current # of weeks left in the selling season
inv = 1421 #current amount of inventory left
sal = 13.87 #salvage value per unit

In [ ]:
sum(demand[i]*week[i]*price[i]) + (inv - sum(demand[i]*week[i]))*sal
sum(demand[i]*week[i]*price[i]) - sum(demand[i]*week[i])*sal + inv*sal


In [ ]:
#calculate discounted prices
price = [list_price, list_price*(1-.15),list_price*(1-.3), list_price*(1-.5)] #for 0%, 15%, 30%, 50% discounts, respectively

#defining the optimization model
model = ConcreteModel()
model.x = Var(range(4), domain=NonNegativeReals)
model.Objective = Objective(expr = sum(demand[i]*model.x[i]*(price[i]-sal) for i in range(4)) + sal*inv, sense=maximize)
model.InvConstraint = Constraint(expr = sum(demand[i]*model.x[i] for i in range(4)) <= inv)
model.WeeksConstraint = Constraint(expr = sum(model.x[i] for i in range(4)) <= weeksleft)
model.pprint()

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {0, 1, 2, 3}

1 Var Declarations
    x : Size=4, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    Objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 4993.2*x[0] + 5326.08*x[1] + 4843.404*x[2] + 3093.0099999999998*x[3] + 19709.27

2 Constraint Declarations
    InvConstraint : Size=1, Index=None, Active=True
        Key  : Lower : Body                                      : Upper  : Active
        None :  -Inf : 120*x[0] + 160*

In [ ]:
#solving
opt = SolverFactory('glpk')
opt.solve(model, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmp95dm1lxd.glpk.raw --wglp /tmp/tmp0mkpoyw4.glpk.glp --cpxlp
 /tmp/tmpe9bm036t.pyomo.lp
Reading problem data from '/tmp/tmpe9bm036t.pyomo.lp'...
3 rows, 5 columns, 9 non-zeros
35 lines were read
Writing problem data to '/tmp/tmp0mkpoyw4.glpk.glp'...
28 lines were written
GLPK Simplex Optimizer, v4.65
3 rows, 5 columns, 9 non-zeros
Preprocessing...
2 rows, 4 columns, 8 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  2.230e+02  ratio =  2.230e+02
GM: min|aij| =  8.565e-01  max|aij| =  1.168e+00  ratio =  1.363e+00
EQ: min|aij| =  7.336e-01  max|aij| =  1.000e+00  ratio =  1.363e+00
Constructing initial basis...
Size of triangular part is 2
*     0: obj =   1.970927000e+04 inf =   0.000e+00 (4)
*     2: obj =   7.883708000e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (40412 bytes)
Writing basic solution to '/tmp/tmp95dm1lxd.glpk.raw'...


{'Problem': [{'Name': 'unknown', 'Lower bound': 78837.08, 'Upper bound': 78837.08, 'Number of objectives': 1, 'Number of constraints': 3, 'Number of variables': 5, 'Number of nonzeros': 9, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.03786492347717285}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [ ]:
#print results
print('optimal scheduled time at each price level over the remaining weeks: ', (model.x[0](), model.x[1](), model.x[2](), model.x[3]()))
print('projected optimal total revenue: ', model.Objective())

optimal scheduled time at each price level over the remaining weeks:  (11.8416666666667, 0.0, 0.0, 0.0)
projected optimal total revenue:  78837.08000000016


#Scenarios

In [4]:
df = pd.read_excel('/content/MP_scenarios.xlsx')

In [5]:
df

,No,Price,Demand (0% discount),Demand (15% discount),Demand (30% discount),Demand (50% discount),Weeks Left,Inventory Remaining,Salvage Value
0,1,55.48,120,160,194,223,15,1421,13.870
1,2,53.68,115,149,171,197,15,2396,13.420
2,3,61.56,140,191,207,459,15,2544,15.390
3,4,65.72,115,151,242,278,15,1316,16.430
4,5,64.98,120,173,221,335,15,1377,16.245
...,...,...,...,...,...,...,...,...,...
195,196,58.18,105,152,160,454,15,2320,14.545
196,197,58.58,125,169,177,298,15,1465,14.645
197,198,50.18,145,218,263,362,15,1433,12.545
198,199,59.58,140,185,224,292,15,1355,14.895


In [6]:
#define a function to obtain the pricing plan and projected revenue for each scenario
def pricing_plan(list_price, demand, weeksleft, inv, sal):
  #calculate discounted prices
  price = [list_price, list_price*(1-.15),list_price*(1-.3), list_price*(1-.5)]

  #defining the optimization model
  model = ConcreteModel()
  model.x = Var(range(4), domain=NonNegativeReals)
  model.Objective = Objective(expr = sum(demand[i]*model.x[i]*(price[i]-sal) for i in range(4)) + sal*inv, sense=maximize)
  model.InvConstraint = Constraint(expr = sum(demand[i]*model.x[i] for i in range(4)) <= inv)
  model.WeeksConstraint = Constraint(expr = sum(model.x[i] for i in range(4)) <= weeksleft)
  #solving
  opt = SolverFactory('glpk')
  opt.solve(model, tee=False)

  return [[model.x[0](), model.x[1](), model.x[2](), model.x[3]()], model.Objective()]

In [7]:
plan = []
revenue = []
for i in range(len(df)):
  plan.append(pricing_plan(df.iloc[i,1], df.iloc[i,2:6].values.tolist(), df.iloc[i,6], df.iloc[i,7], df.iloc[i,8])[0])
  revenue.append(pricing_plan(df.iloc[i,1], df.iloc[i,2:6].values.tolist(), df.iloc[i,6], df.iloc[i,7], df.iloc[i,8])[1])

In [8]:
df['pricing_plan'] = plan
df['projected_revenue'] = revenue

In [9]:
df

,No,Price,Demand (0% discount),Demand (15% discount),Demand (30% discount),Demand (50% discount),Weeks Left,Inventory Remaining,Salvage Value,pricing_plan,projected_revenue
0,1,55.48,120,160,194,223,15,1421,13.870,"[11.8416666666667, 0.0, 0.0, 0.0]",78837.080000
1,2,53.68,115,149,171,197,15,2396,13.420,"[0.0, 15.0, 0.0, 0.0]",104139.200000
2,3,61.56,140,191,207,459,15,2544,15.390,"[6.29411764705882, 8.70588235294118, 0.0, 0.0]",141254.127529
3,4,65.72,115,151,242,278,15,1316,16.430,"[11.4434782608696, 0.0, 0.0, 0.0]",86487.520000
4,5,64.98,120,173,221,335,15,1377,16.245,"[11.475, 0.0, 0.0, 0.0]",89477.460000
...,...,...,...,...,...,...,...,...,...,...,...
195,196,58.18,105,152,160,454,15,2320,14.545,"[0.0, 14.8675496688742, 0.0, 0.132450331125827]",113506.482914
196,197,58.58,125,169,177,298,15,1465,14.645,"[11.72, 0.0, 0.0, 0.0]",85819.700000
197,198,50.18,145,218,263,362,15,1433,12.545,"[9.88275862068966, 0.0, 0.0, 0.0]",71907.940000
198,199,59.58,140,185,224,292,15,1355,14.895,"[9.67857142857143, 0.0, 0.0, 0.0]",80730.900000


In [10]:
df.to_csv('Solutions.csv')